### 37. 텐서를 다루다

#### 1) 원소별 계산

In [21]:
import numpy as np
import dezero.functions as F
from dezero import Variable
from dezero import Function

In [7]:
x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

variable(0.8414709848078965)


In [8]:
# x가 텐서인 경우, sin함수가 원소별로 적용됨 (broadcast)

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


#### 2) 텐서 사용시의 역전파
- 그동안 '스칼라'를 대상으로 역전파를 구현함. 브로드캐스팅에 의해 역전파는 '텐서'의 원소별 계산에서도 성립

<img src='img/37_1.png' width='400'>

In [9]:
# 확인

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
c = Variable(np.array([[10, 20, 30], [40, 50, 60]]))
t = x + c
y = F.sum(t)

In [10]:
y.backward(retain_grad=True)
print(y.grad)
print(t.grad)
print(x.grad)
print(c.grad)

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


현재, 기울기의 형상과 데이터의 형상이 일치 (즉, x.shape == x.grad.shape)  
-> sum이나 reshape 함수 등을 구현하기 어렵지 않음

#### 3) [보충] 텐서 사용 시의 역전파
합성 함수 __$y$__ = $F($__$y$__$)$, __$a$__ = $A($__$x$__$)$, __$b$__ = $B($__$a$__$)$, __$y$__ = $C($__$b$__$)$에서 y의 __$x$__에 대한 미분은 아래와 같다. 계산순서 forward 모드와 reverse 모드를 살펴보자.
<img src='img/37_2.png' width='100'>
 
- forward 모드
<img src='img/37_3.png' width='300'>

- reverse 모드
<img src='img/37_4.png' width='280'>


- 이때, reverse 모드는 1\*n 벡터를 전파하지만, forward 모드에서는 n\*n을 전파한다. 계산량이 적으므로 계산 효율이 좋다.
- 또한, 야코비안 행렬을 구하여 '행렬의 곱'을 계산할 필요가 없다. $\partial a/\partial x$이 대각행렬이기 때문에 원소별로 곱하면 된다.
<img src='img/37_5.png' width='120'>

### 38. 형상 변환 함수
- 원소별 연산을 수행하는 함수(add, sin 등)는 텐서여도 잘 계산할 수 있지만,  
reshape과 transpose 함수처럼 텐서의 형상을 변환하는 함수들은 다시 구현을 해주어야 한다

#### 1) reshape 함수 구현
<img src='img/37_6.png' width='250'>

In [1]:
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.reshape(x, (6, ))
print(y)

[1 2 3 4 5 6]


In [22]:
# dezero의 functions 파일에 추가해주기

class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape
        
    def forward(self, x):
        self.x_shape = x.shape   # reshape 하기 전의 shape
        y = x.reshape(self.shape)
        return y
    
    def backward(self, gy):
        return reshape(gy, self.x_shape)  # reshape 하기 전의 shape로 돌려주기

In [23]:
from dezero.core import as_variable

def reshape(x, shape):
    if x.shape == shape:  # reshape하려는 shape과 현재의 shape이 같으면
        return as_variable(x)  # 그대로 돌려주기
    return Reshape(shape)(x)

In [24]:
# 구현한 reshape 함수 사용하기

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.reshape(x, (6, ))
y.backward(retain_grad=True)
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


#### 2) Variable에서 reshape 사용하기

In [26]:
x = np.random.rand(1, 2, 3)

y = x.reshape((2, 3))
y = x.reshape([2, 3])
y = x.reshape(2, 3)

In [25]:
# dezero의 core파일에서 Variable 클래스에 추가하기

def reshape(self, *shape):
    if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
        shape = shape[0]
    
    return dezero.functions.reshape(self, shape)

In [27]:
x = Variable(np.random.randn(1, 2, 3))
y = x.reshape((2, 3))
y = x.reshape(2, 3)

#### 3) 행렬의 전치

In [29]:
# 넘파이에서의 transpose

x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


In [30]:
# dezero의 functions 파일에 추가하기

class Transpose(Function):
    def forward(self, x):
        y = np.transpose(x)
        return y
    
    def backward(self, gy):
        gx = transpose(gy)
        return gx
    
def transpose(x):
    return Transpose()(x)

In [31]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.transpose(x)
y.backward()
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


In [32]:
# dezero의 core 파일에서 Variable 클래스에 추가하기

def transpose(self):
    return dezero.functions.transpose(self)

@property
def T(self):
    return dezero.functions.transpose(self)

In [33]:
x = Variable(np.random.rand(2, 3))
y = x.transpose()
y = x.T

#### 4) 실제 transpose 함수
- 넘파이의 np.transpose 함수는 더 범용적으로 사용가능

In [37]:
# 축의 데이터 순서 바꿀 수 있음

A, B, C, D = 1, 2, 3, 4
x = np.random.rand(A, B, C, D)
x

array([[[[0.59011832, 0.0598224 , 0.18197858, 0.84619198],
         [0.03845406, 0.80756561, 0.21671976, 0.80517591],
         [0.52603176, 0.62795826, 0.76339484, 0.06938922]],

        [[0.96843249, 0.20920264, 0.87448685, 0.61155848],
         [0.67926472, 0.21911552, 0.00572954, 0.71833413],
         [0.31517635, 0.17452025, 0.15189772, 0.03867657]]]])

In [38]:
y = x.transpose(1, 0, 3, 2)  # 인수에 아무것도 넣지 않으면 역순으로 정렬됨
y                            # 0번째와 1번째 축의 데이터 -> 1번째와 0번째 순서

array([[[[0.59011832, 0.03845406, 0.52603176],
         [0.0598224 , 0.80756561, 0.62795826],
         [0.18197858, 0.21671976, 0.76339484],
         [0.84619198, 0.80517591, 0.06938922]]],


       [[[0.96843249, 0.67926472, 0.31517635],
         [0.20920264, 0.21911552, 0.17452025],
         [0.87448685, 0.00572954, 0.15189772],
         [0.61155848, 0.71833413, 0.03867657]]]])

In [35]:
x.shape

(1, 2, 3, 4)

In [36]:
y.shape

(2, 1, 4, 3)